In [1]:
from tensorflow.python.keras import backend
from tensorflow.python.keras.applications import imagenet_utils
from tensorflow.python.keras.engine import training
from tensorflow.python.keras.layers import VersionAwareLayers
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.python.keras.utils import data_utils
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.lib.io import file_io
from tensorflow.python.util.tf_export import keras_export
from tensorflow.keras import datasets
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np

In [ ]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train = tf.keras.applications.resnet50.preprocess_input(x_train.astype(np.float32))
x_test = tf.keras.applications.resnet50.preprocess_input(x_test.astype(np.float32))

In [ ]:
def define_resnet50():
    inputs = tf.keras.layers.Input(shape=(32, 32, 3))
    resized = tf.keras.layers.UpSampling2D(size=(7, 7))(inputs)

    features = tf.keras.applications.ResNet50(input_shape=(224, 224, 3),
                                              include_top=False,
                                              weights='imagenet')(resized)

    x = tf.keras.layers.GlobalAveragePooling2D()(features)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)

    return model


model = define_resnet50()
model.summary()

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
optim = optimizers.RMSprop(centered=True, learning_rate=0.0001)
# optim = optimizers.Adam(learning_rate=0.001, amsgrad=True)
# optim=optimizers.SGD(learning_rate=0.0001,nesterov=True)
# optim=optimizers.Adadelta(learning_rate=0.001)
model.compile(optimizer=optim, metrics=['accuracy'], loss='sparse_categorical_crossentropy')

In [ ]:
model.fit(x_train, y_train, batch_size=90, epochs=100, validation_data=(x_test, y_test), shuffle=True,
          callbacks=[model_checkpoint_callback])